## Linear Regression

In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName('MAST30034 Tutorial 3')
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '4g')
    .config('spark.executor.memory', '2g')
    .getOrCreate()
)

24/08/14 10:49:23 WARN Utils: Your hostname, MacBook-Pro-7.local resolves to a loopback address: 127.0.0.1; using 10.12.165.57 instead (on interface en0)
24/08/14 10:49:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/14 10:49:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
sdf = spark.read.parquet("../data/tlc_data/raw/*")

In [4]:
# VectorAssembler creates new vectors from existing columns
from pyspark.ml.feature import VectorAssembler

features = 'features'
input_cols = ['driver_pay', 'tips', 'tolls', 'trip_miles']

assembler = VectorAssembler(
    # which column to combine
    inputCols=input_cols, 
    # How should the combined columns be named
    outputCol=features
)

model_sdf = assembler.transform(sdf.dropna('any'))
# Display the features and targets for our model
model_sdf.select('features').head(5), model_sdf.select('driver_pay').head(5)

([Row(features=DenseVector([27.83, 5.22, 0.0, 0.94])),
  Row(features=DenseVector([50.15, 0.0, 0.0, 2.78])),
  Row(features=DenseVector([20.22, 0.0, 0.0, 8.81])),
  Row(features=DenseVector([7.9, 0.0, 0.0, 0.67])),
  Row(features=DenseVector([16.48, 0.0, 0.0, 4.38]))],
 [Row(driver_pay=27.83),
  Row(driver_pay=50.15),
  Row(driver_pay=20.22),
  Row(driver_pay=7.9),
  Row(driver_pay=16.48)])

In [5]:
from pyspark.ml.regression import LinearRegression

lm = LinearRegression(
    featuresCol='features', 
    labelCol='driver_pay'
).fit(model_sdf)

24/08/14 10:49:52 WARN Instrumentation: [232037ab] regParam is zero, which might cause numerical instability and overfitting.
24/08/14 10:49:53 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/08/14 10:49:53 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
24/08/14 10:50:30 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [8]:
import pandas as pd

# Access coefficients
pd.DataFrame(
    data=[lm.intercept] + list(lm.coefficients),
    index=['intercept'] + input_cols,
    columns=['coefficient']
)

,coefficient
intercept,1.924876e-14
driver_pay,1.000000e+00
tips,8.623112e-16
tolls,-1.540403e-16
trip_miles,8.145676e-15


In [9]:
# example record to predict
sdf.select('driver_pay', *input_cols).limit(1).show(vertical=True)

-RECORD 0-----------
 driver_pay | 27.83 
 driver_pay | 27.83 
 tips       | 5.22  
 tolls      | 0.0   
 trip_miles | 0.94  



In [10]:
# preprocess for predictions
predict_test = sdf.select(*input_cols).limit(1)

assembler = VectorAssembler(
    inputCols=input_cols, 
    outputCol=features
)

predict_sdf = assembler.transform(predict_test).select(features)

predict_sdf.show(1, vertical=True)

-RECORD 0------------------------
 features | [27.83,5.22,0.0,0... 



In [11]:
predictions = lm.transform(predict_sdf)
predictions.show(vertical=True, truncate=False)

-RECORD 0---------------------------
 features   | [27.83,5.22,0.0,0.94] 
 prediction | 27.829999999999945    



In [12]:
# r2 example
lm.summary.r2

1.0

24/08/14 14:28:12 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 554673 ms exceeds timeout 120000 ms
24/08/14 14:28:12 WARN SparkContext: Killing executors is not supported by current scheduler.
24/08/14 14:28:15 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$